# **INTRODUCTIONS**

---

This flow is used to map out the diverse COVID-19 variants that are discovered.

2022-04: Added this flow because of a dependency is needed for ecdc_variants

1. **[Stored Procedures](#stored=procedures)**


# **DEPENDENCIES**
---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/utils/conversions.ipynb",
        "src/utils/protos.ipynb",
        "src/statics/masterdata_variants.ipynb"
    ]
}
```

# **TABLES**
---

### STAGINGS

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

IF NOT EXISTS(SELECT * FROM sys.sequences WHERE object_id = OBJECT_ID(N'[dbo].[SEQ_VWSSTAGE_RIVM_MUTATIONS]') AND type = 'SO')
CREATE SEQUENCE [SEQ_VWSSTAGE_RIVM_MUTATIONS]
  START WITH 1
  INCREMENT BY 1;
GO

IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[RIVM_MUTATIONS]'))
BEGIN
CREATE TABLE [VWSSTAGE].[RIVM_MUTATIONS] (
    [ID]                            INT           DEFAULT (NEXT VALUE FOR [dbo].[SEQ_VWSSTAGE_RIVM_MUTATIONS]) NOT NULL,
    [VERSION]                       VARCHAR (100) NULL,
    [DATE_OF_REPORT]                VARCHAR (100) NULL,
    [DATE_OF_STATISTICS_WEEK_START] VARCHAR (100) NULL,
    [VARIANT_CODE]                  VARCHAR (100) NULL,
    [VARIANT_NAME]                  VARCHAR (100) NULL,
    [ECDC_CATEGORY]                 VARCHAR (100) NULL,
    [SAMPLE_SIZE]                   VARCHAR (100) NULL,
    [VARIANT_CASES]                 VARCHAR (100) NULL,
    [DATE_LAST_INSERTED]            DATETIME      DEFAULT (getdate()) NULL,
    PRIMARY KEY CLUSTERED ([ID] ASC)
);
END


IF NOT EXISTS(SELECT * FROM sys.indexes WHERE NAME='IX_STAGE_RIVM_MUTATIONS')
BEGIN
CREATE NONCLUSTERED INDEX [IX_STAGE_RIVM_MUTATIONS]
    ON [VWSSTAGE].[RIVM_MUTATIONS]([DATE_LAST_INSERTED] ASC)
    INCLUDE([DATE_OF_REPORT], [DATE_OF_STATISTICS_WEEK_START], [VARIANT_CODE], [VARIANT_NAME], [ECDC_CATEGORY], [SAMPLE_SIZE], [VARIANT_CASES]);
END



### INTERMEDIATES

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.
IF NOT EXISTS(SELECT * FROM sys.sequences WHERE object_id = OBJECT_ID(N'[dbo].[SEQ_VWSINTER_RIVM_MUTATIONS]') AND type = 'SO')
CREATE SEQUENCE SEQ_VWSINTER_RIVM_MUTATIONS
  START WITH 1
  INCREMENT BY 1;
GO

IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSINTER].[RIVM_MUTATIONS]'))
BEGIN
CREATE TABLE [VWSINTER].[RIVM_MUTATIONS] (
    [ID]                            INT           DEFAULT (NEXT VALUE FOR [dbo].[SEQ_VWSINTER_RIVM_MUTATIONS]) NOT NULL,
    [VERSION]                       INT           NULL,
    [DATE_OF_REPORT]                DATETIME      NULL,
    [DATE_OF_STATISTICS_WEEK_START] DATETIME      NULL,
    [VARIANT_CODE]                  VARCHAR (100) NULL,
    [VARIANT_NAME]                  VARCHAR (100) NULL,
    [ECDC_CATEGORY]                 VARCHAR (100) NULL,
    [SAMPLE_SIZE]                   INT           NULL,
    [VARIANT_CASES]                 INT           NULL,
    [DATE_LAST_INSERTED]            DATETIME      DEFAULT (getdate()) NULL,
    PRIMARY KEY CLUSTERED ([ID] ASC)
);
END

IF NOT EXISTS(SELECT * FROM sys.indexes WHERE NAME='IX_INTER_RIVM_MUTATIONS')
CREATE NONCLUSTERED INDEX [IX_INTER_RIVM_MUTATIONS]
    ON [VWSINTER].[RIVM_MUTATIONS]([DATE_LAST_INSERTED] ASC)
    INCLUDE([DATE_OF_REPORT], [DATE_OF_STATISTICS_WEEK_START], [VARIANT_CODE], [VARIANT_NAME], [ECDC_CATEGORY], [SAMPLE_SIZE], [VARIANT_CASES]);
GO

### DESTINATIONS

In [ ]:
-- Copyright (c) 2021 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.
IF NOT EXISTS(SELECT * FROM sys.sequences WHERE object_id = OBJECT_ID(N'[dbo].[SEQ_VWSDEST_RIVM_MUTATIONS]') AND type = 'SO')
CREATE SEQUENCE SEQ_VWSDEST_RIVM_MUTATIONS
  START WITH 1
  INCREMENT BY 1;
GO

IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[RIVM_MUTATIONS]'))
BEGIN
CREATE TABLE [VWSDEST].[RIVM_MUTATIONS] (
    [ID]                            INT             DEFAULT (NEXT VALUE FOR [dbo].[SEQ_VWSDEST_RIVM_MUTATIONS]) NOT NULL,
    [DATE_OF_REPORT]                DATETIME        NULL,
    [DATE_OF_STATISTICS_WEEK_START] DATETIME        NULL,
    [VARIANT_CODE]                  VARCHAR (100)   NULL,
    [VARIANT_NAME]                  VARCHAR (100)   NULL,
    [IS_VARIANT_OF_CONCERN]         VARCHAR (100)   NULL,
    [SAMPLE_SIZE]                   INT             NULL,
    [VARIANT_CASES]                 INT             NULL,
    [VARIANT_PERCENTAGE]            DECIMAL (16, 3) NULL,
    [DATE_LAST_INSERTED]            DATETIME        DEFAULT (getdate()) NULL,
    [HAS_HISTORICAL_SIGNIFICANCE]   VARCHAR (100)   NULL,
    [LABEL_EN] VARCHAR (100) NULL,
    [LABEL_NL] VARCHAR (100) NULL,
    [SORT_ORDER] INT NULL,
    PRIMARY KEY CLUSTERED ([ID] ASC)
);
END
ELSE
BEGIN
	IF NOT EXISTS(SELECT * FROM SYS.COLUMNS 
        WHERE [NAME] IN (
            N'LABEL_EN', 
            N'LABEL_NL', 
            N'SORT_ORDER'
            )
        AND [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[RIVM_MUTATIONS]'))
	BEGIN
		ALTER TABLE [VWSDEST].[RIVM_MUTATIONS] 
		ADD 
            [LABEL_EN] VARCHAR (100) NULL,
            [LABEL_NL] VARCHAR (100) NULL,
            [SORT_ORDER] INT NULL;
	END
END



IF NOT EXISTS(SELECT * FROM sys.indexes WHERE NAME='IX_DEST_RIVM_MUTATIONS')
   CREATE NONCLUSTERED INDEX [IX_DEST_RIVM_MUTATIONS]
    ON [VWSDEST].[RIVM_MUTATIONS]([DATE_LAST_INSERTED] ASC)
    INCLUDE([DATE_OF_REPORT], [DATE_OF_STATISTICS_WEEK_START], [VARIANT_CODE], [VARIANT_NAME], [IS_VARIANT_OF_CONCERN], [SAMPLE_SIZE], [VARIANT_CASES], [VARIANT_PERCENTAGE]);
GO

# **VIEWS**
---

#### V_RIVM_MUTATIONS

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

--File name has _A_ to make sure this file is executed before the view that depends on this.
CREATE OR ALTER  VIEW [VWSDEST].[V_RIVM_MUTATIONS]
AS

WITH BASE_CTE AS (
SELECT [ID]
      ,[DATE_OF_REPORT]
      ,[DATE_OF_STATISTICS_WEEK_START]
      ,[VARIANT_CODE]
      ,[LABEL_EN]
      ,[LABEL_NL]
      ,[SORT_ORDER] AS [INDEX]
      ,[VARIANT_NAME]
      --The following two columns will be used to turn into column names using pivot.
      ,[VARIANT_NAME] + '_IS_VARIANT_OF_CONCERN' AS [PIVOT_IS_VARIANT_OF_CONCERN]
      ,[VARIANT_NAME] + '_OCCURRENCE' AS [PIVOT_OCCURRENCE]
      ,[IS_VARIANT_OF_CONCERN]
      ,[SAMPLE_SIZE]
      ,[VARIANT_CASES] AS [OCCURRENCE]
      ,[VARIANT_PERCENTAGE] AS [PERCENTAGE]
      ,[DATE_LAST_INSERTED]
  FROM [VWSDEST].[RIVM_MUTATIONS]
  WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[RIVM_MUTATIONS])
)
,
--This CTE is used to explicitly retrieve the sample size per week.
SAMPLE_SIZE_CTE AS (
    SELECT 
           MAX([SAMPLE_SIZE]) AS [SAMPLE_SIZE] --Only one value present
          ,[DATE_OF_REPORT] 
          ,[DATE_OF_STATISTICS_WEEK_START] 
    FROM BASE_CTE
        GROUP BY [DATE_OF_REPORT] ,[DATE_OF_STATISTICS_WEEK_START]
)
,
--CTE pivotting if the variant is of concern per variant
VARIANT_CTE AS (

    SELECT 
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS_WEEK_START],
        [DATE_LAST_INSERTED],  
        Alpha_IS_VARIANT_OF_CONCERN,
        Beta_IS_VARIANT_OF_CONCERN,
        Gamma_IS_VARIANT_OF_CONCERN,
        Delta_IS_VARIANT_OF_CONCERN,
        Epsilon_IS_VARIANT_OF_CONCERN,
        Eta_IS_VARIANT_OF_CONCERN,
        Theta_IS_VARIANT_OF_CONCERN,
        Kappa_IS_VARIANT_OF_CONCERN,
        Lambda_IS_VARIANT_OF_CONCERN,
        Other_IS_VARIANT_OF_CONCERN
    FROM
        (
            SELECT 
                [DATE_OF_REPORT],
                [DATE_OF_STATISTICS_WEEK_START],
                [DATE_LAST_INSERTED],  
                [PIVOT_IS_VARIANT_OF_CONCERN],
                [IS_VARIANT_OF_CONCERN]
            FROM BASE_CTE
        ) AS PIVOT_DATA
        PIVOT (
                    MAX([IS_VARIANT_OF_CONCERN]) --Only one value present
                    FOR [PIVOT_IS_VARIANT_OF_CONCERN] IN (
                        Alpha_IS_VARIANT_OF_CONCERN,
                        Beta_IS_VARIANT_OF_CONCERN,
                        Gamma_IS_VARIANT_OF_CONCERN,
                        Delta_IS_VARIANT_OF_CONCERN,
                        Epsilon_IS_VARIANT_OF_CONCERN,
                        Eta_IS_VARIANT_OF_CONCERN,
                        Theta_IS_VARIANT_OF_CONCERN,
                        Kappa_IS_VARIANT_OF_CONCERN,
                        Lambda_IS_VARIANT_OF_CONCERN,
                        Other_IS_VARIANT_OF_CONCERN)
            ) AS P1
)
,
--CTE pivotting the OCCURRENCE numbers per variant
OCCURRENCE_CTE AS (
    SELECT 
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS_WEEK_START],  
        Alpha_OCCURRENCE,
        Beta_OCCURRENCE,
        Gamma_OCCURRENCE,
        Delta_OCCURRENCE,
        Epsilon_OCCURRENCE,
        Eta_OCCURRENCE,
        Theta_OCCURRENCE,
        Kappa_OCCURRENCE,
        Lambda_OCCURRENCE,
        Other_OCCURRENCE
    FROM
        (
            SELECT 
                [DATE_OF_REPORT],
                [DATE_OF_STATISTICS_WEEK_START],
                [PIVOT_OCCURRENCE],
                [OCCURRENCE]
            FROM BASE_CTE
        ) AS PIVOT_DATA
        PIVOT (
                    MAX([OCCURRENCE]) --Only one value present
                    FOR [PIVOT_OCCURRENCE] IN (
                        Alpha_OCCURRENCE,
                        Beta_OCCURRENCE,
                        Gamma_OCCURRENCE,
                        Delta_OCCURRENCE,
                        Epsilon_OCCURRENCE,
                        Eta_OCCURRENCE,
                        Theta_OCCURRENCE,
                        Kappa_OCCURRENCE,
                        Lambda_OCCURRENCE,
                        Other_OCCURRENCE)
            ) AS P1
)


--Final selection

--Lambda is set to NULL as it is not included in the selection (that determines the size of other_variants)
SELECT 
        [dbo].[CONVERT_DATETIME_TO_UNIX](T0.DATE_OF_STATISTICS_WEEK_START) AS DATE_START_UNIX,
        [dbo].[CONVERT_DATETIME_TO_UNIX](dbo.WEEK_END(T0.DATE_OF_STATISTICS_WEEK_START)) AS DATE_END_UNIX,
        ISNULL(T0.Alpha_IS_VARIANT_OF_CONCERN,  'false') AS Alpha_IS_VARIANT_OF_CONCERN,      
        ISNULL(T0.Beta_IS_VARIANT_OF_CONCERN,   'false') AS Beta_IS_VARIANT_OF_CONCERN,       
        ISNULL(T0.Gamma_IS_VARIANT_OF_CONCERN,  'false') AS Gamma_IS_VARIANT_OF_CONCERN,      
        ISNULL(T0.Delta_IS_VARIANT_OF_CONCERN,  'false') AS Delta_IS_VARIANT_OF_CONCERN,      
        ISNULL(T0.Epsilon_IS_VARIANT_OF_CONCERN,'false') AS Epsilon_IS_VARIANT_OF_CONCERN,        
        ISNULL(T0.Eta_IS_VARIANT_OF_CONCERN,    'false') AS Eta_IS_VARIANT_OF_CONCERN,            
        ISNULL(T0.Theta_IS_VARIANT_OF_CONCERN,  'false') AS Theta_IS_VARIANT_OF_CONCERN,          
        ISNULL(T0.Kappa_IS_VARIANT_OF_CONCERN,  'false') AS Kappa_IS_VARIANT_OF_CONCERN,          
        -- ISNULL(T0.Lambda_IS_VARIANT_OF_CONCERN, 'false') AS Lambda_IS_VARIANT_OF_CONCERN,    
        NULL AS      Lambda_IS_VARIANT_OF_CONCERN,
        ISNULL(T0.Other_IS_VARIANT_OF_CONCERN,  'false') AS Other_IS_VARIANT_OF_CONCERN,          
        ISNULL(T1.Alpha_OCCURRENCE,             0) AS Alpha_OCCURRENCE,                             
        ISNULL(T1.Beta_OCCURRENCE,              0) AS Beta_OCCURRENCE,                          
        ISNULL(T1.Gamma_OCCURRENCE,             0) AS Gamma_OCCURRENCE,                             
        ISNULL(T1.Delta_OCCURRENCE,             0) AS Delta_OCCURRENCE,                     
        ISNULL(T1.Epsilon_OCCURRENCE,           0) AS Epsilon_OCCURRENCE,                           
        ISNULL(T1.Eta_OCCURRENCE,               0) AS Eta_OCCURRENCE,                               
        ISNULL(T1.Theta_OCCURRENCE,             0) AS Theta_OCCURRENCE,                         
        ISNULL(T1.Kappa_OCCURRENCE,             0) AS Kappa_OCCURRENCE,                                 
        -- ISNULL(T1.Lambda_OCCURRENCE,            0) AS Lambda_OCCURRENCE,       
        NULL AS      Lambda_OCCURRENCE,                       
        ISNULL(T1.Other_OCCURRENCE,             0) AS Other_OCCURRENCE,                         
        --Calculate the percentage by using sample size.
        --Recalculating it here requires less code than another pivot.
        ISNULL(ROUND(CAST(T1.Alpha_OCCURRENCE     * 100 AS FLOAT) / T2.[SAMPLE_SIZE],1),0) AS ALPHA_PERCENTAGE,
        ISNULL(ROUND(CAST(T1.Beta_OCCURRENCE      * 100 AS FLOAT) / T2.[SAMPLE_SIZE],1),0) AS BETA_PERCENTAGE,
        ISNULL(ROUND(CAST(T1.Gamma_OCCURRENCE     * 100 AS FLOAT) / T2.[SAMPLE_SIZE],1),0) AS GAMMA_PERCENTAGE,
        ISNULL(ROUND(CAST(T1.Delta_OCCURRENCE     * 100 AS FLOAT) / T2.[SAMPLE_SIZE],1),0) AS DELTA_PERCENTAGE,
        ISNULL(ROUND(CAST(T1.Epsilon_OCCURRENCE   * 100 AS FLOAT) / T2.[SAMPLE_SIZE],1),0) AS EPSILON_PERCENTAGE,
        ISNULL(ROUND(CAST(T1.Eta_OCCURRENCE       * 100 AS FLOAT) / T2.[SAMPLE_SIZE],1),0) AS ETA_PERCENTAGE,
        ISNULL(ROUND(CAST(T1.Theta_OCCURRENCE     * 100 AS FLOAT) / T2.[SAMPLE_SIZE],1),0) AS THETA_PERCENTAGE,
        ISNULL(ROUND(CAST(T1.Kappa_OCCURRENCE     * 100 AS FLOAT) / T2.[SAMPLE_SIZE],1),0) AS KAPPA_PERCENTAGE,
        -- ISNULL(ROUND(CAST(T1.Lambda_OCCURRENCE    * 100 AS FLOAT) / T2.[SAMPLE_SIZE],1),0) AS LAMBDA_PERCENTAGE,
        NULL AS      LAMBDA_PERCENTAGE,
        ISNULL(ROUND(CAST(T1.Other_OCCURRENCE     * 100 AS FLOAT) / T2.[SAMPLE_SIZE],1),0) AS OTHER_PERCENTAGE,

        T2.SAMPLE_SIZE,
        [dbo].[CONVERT_DATETIME_TO_UNIX](T0.DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM VARIANT_CTE T0
    LEFT JOIN OCCURRENCE_CTE T1 ON T0.[DATE_OF_REPORT] = T1.[DATE_OF_REPORT] AND T0.[DATE_OF_STATISTICS_WEEK_START] = T1.[DATE_OF_STATISTICS_WEEK_START]
    LEFT JOIN SAMPLE_SIZE_CTE T2 ON T0.[DATE_OF_REPORT] = T2.[DATE_OF_REPORT] AND T0.[DATE_OF_STATISTICS_WEEK_START] = T2.[DATE_OF_STATISTICS_WEEK_START]
GO


#### V_RIVM_MUTATIONS_TABULAR

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

--File name has _A_ to make sure this file is executed before the view that depends on this.
CREATE OR ALTER  VIEW [VWSDEST].[V_RIVM_MUTATIONS_TABULAR]
AS
SELECT 
    --   ,[DATE_OF_REPORT]
       [dbo].[CONVERT_DATETIME_TO_UNIX](DATE_OF_STATISTICS_WEEK_START) AS DATE_START_UNIX
      ,[dbo].[CONVERT_DATETIME_TO_UNIX](dbo.WEEK_END(DATE_OF_STATISTICS_WEEK_START)) AS DATE_END_UNIX
      ,[VARIANT_CODE]
      ,[LABEL_EN] as [VARIANT_NAME_LABEL_EN]
      ,[LABEL_NL] as [NAME]
      ,[SORT_ORDER] AS [INDEX]
      ,[IS_VARIANT_OF_CONCERN]
      ,[HAS_HISTORICAL_SIGNIFICANCE]
      ,[SAMPLE_SIZE]
      ,[VARIANT_CASES] AS [OCCURRENCE]
      ,[VARIANT_PERCENTAGE] AS [PERCENTAGE]
      ,[dbo].[CONVERT_DATETIME_TO_UNIX](DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
  FROM [VWSDEST].[RIVM_MUTATIONS]
  WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[RIVM_MUTATIONS])
GO


#### [V_DIFFERENCES_MUTATIONS__PERCENTAGE_ALPHA]

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

--All views required for the variant differences are created here

CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCES_MUTATIONS__PERCENTAGE_ALPHA]
AS
WITH BASE_CTE AS (
SELECT [DATE_START_UNIX]
      ,LAG(DATE_START_UNIX,1) OVER (ORDER BY DATE_START_UNIX) AS OLD_DATE_UNIX
      ,[ALPHA_PERCENTAGE] AS NEW_VALUE
      ,LAG(ALPHA_PERCENTAGE,1) OVER (ORDER BY DATE_START_UNIX) AS OLD_VALUE
      ,ROW_NUMBER() OVER (ORDER BY DATE_START_UNIX DESC) AS RANK_DATE
  FROM [VWSDEST].[V_RIVM_MUTATIONS]
  --Date_last_inserted filter is not needed since this is a view on a view.
)
SELECT
     OLD_VALUE
    -- ,NEW_VALUE
    ,ROUND(NEW_VALUE - OLD_VALUE,1) AS [DIFFERENCE]
    ,DATE_START_UNIX AS [NEW_DATE_UNIX]
    ,OLD_DATE_UNIX as [OLD_DATE_UNIX]
FROM BASE_CTE
  WHERE RANK_DATE = 1 --Only the most recent record

#### [V_DIFFERENCES_MUTATIONS__PERCENTAGE_BETA]

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCES_MUTATIONS__PERCENTAGE_BETA]
AS
WITH BASE_CTE AS (
SELECT [DATE_START_UNIX]
      ,LAG(DATE_START_UNIX,1) OVER (ORDER BY DATE_START_UNIX) AS OLD_DATE_UNIX
      ,[BETA_PERCENTAGE] AS NEW_VALUE
      ,LAG(BETA_PERCENTAGE,1) OVER (ORDER BY DATE_START_UNIX) AS OLD_VALUE
      ,ROW_NUMBER() OVER (ORDER BY DATE_START_UNIX DESC) AS RANK_DATE
  FROM [VWSDEST].[V_RIVM_MUTATIONS]
  --Date_last_inserted filter is not needed since this is a view on a view.
)
SELECT  
     OLD_VALUE
    -- ,NEW_VALUE
    ,ROUND(NEW_VALUE - OLD_VALUE,1) AS [DIFFERENCE]
    ,DATE_START_UNIX AS [NEW_DATE_UNIX]
    ,OLD_DATE_UNIX as [OLD_DATE_UNIX]
FROM BASE_CTE
  WHERE RANK_DATE = 1 --Only the most recent record

#### [V_DIFFERENCES_MUTATIONS__PERCENTAGE_DELTA]

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCES_MUTATIONS__PERCENTAGE_DELTA]
AS
WITH BASE_CTE AS (
SELECT [DATE_START_UNIX]
      ,LAG(DATE_START_UNIX,1) OVER (ORDER BY DATE_START_UNIX) AS OLD_DATE_UNIX
      ,[DELTA_PERCENTAGE] AS NEW_VALUE
      ,LAG(DELTA_PERCENTAGE,1) OVER (ORDER BY DATE_START_UNIX) AS OLD_VALUE
      ,ROW_NUMBER() OVER (ORDER BY DATE_START_UNIX DESC) AS RANK_DATE
  FROM [VWSDEST].[V_RIVM_MUTATIONS]
  --Date_last_inserted filter is not needed since this is a view on a view.
)
SELECT  
     OLD_VALUE
    -- ,NEW_VALUE
    ,ROUND(NEW_VALUE - OLD_VALUE,1) AS [DIFFERENCE]
    ,DATE_START_UNIX AS [NEW_DATE_UNIX]
    ,OLD_DATE_UNIX as [OLD_DATE_UNIX]
FROM BASE_CTE
  WHERE RANK_DATE = 1 --Only the most recent record

#### [V_DIFFERENCES_MUTATIONS__PERCENTAGE_EPSILON]

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCES_MUTATIONS__PERCENTAGE_EPSILON]
AS
WITH BASE_CTE AS (
SELECT [DATE_START_UNIX]
      ,LAG(DATE_START_UNIX,1) OVER (ORDER BY DATE_START_UNIX) AS OLD_DATE_UNIX
      ,[EPSILON_PERCENTAGE] AS NEW_VALUE
      ,LAG(EPSILON_PERCENTAGE,1) OVER (ORDER BY DATE_START_UNIX) AS OLD_VALUE
      ,ROW_NUMBER() OVER (ORDER BY DATE_START_UNIX DESC) AS RANK_DATE
  FROM [VWSDEST].[V_RIVM_MUTATIONS]
  --Date_last_inserted filter is not needed since this is a view on a view.
)
SELECT  
     OLD_VALUE
    -- ,NEW_VALUE
    ,ROUND(NEW_VALUE - OLD_VALUE,1) AS [DIFFERENCE]
    ,DATE_START_UNIX AS [NEW_DATE_UNIX]
    ,OLD_DATE_UNIX as [OLD_DATE_UNIX]
FROM BASE_CTE
  WHERE RANK_DATE = 1 --Only the most recent record

#### [V_DIFFERENCES_MUTATIONS__PERCENTAGE_ETA]

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCES_MUTATIONS__PERCENTAGE_ETA]
AS
WITH BASE_CTE AS (
SELECT [DATE_START_UNIX]
      ,LAG(DATE_START_UNIX,1) OVER (ORDER BY DATE_START_UNIX) AS OLD_DATE_UNIX
      ,[ETA_PERCENTAGE] AS NEW_VALUE
      ,LAG(ETA_PERCENTAGE,1) OVER (ORDER BY DATE_START_UNIX) AS OLD_VALUE
      ,ROW_NUMBER() OVER (ORDER BY DATE_START_UNIX DESC) AS RANK_DATE
  FROM [VWSDEST].[V_RIVM_MUTATIONS]
  --Date_last_inserted filter is not needed since this is a view on a view.
)
SELECT  
     OLD_VALUE
    -- ,NEW_VALUE
    ,ROUND(NEW_VALUE - OLD_VALUE,1) AS [DIFFERENCE]
    ,DATE_START_UNIX AS [NEW_DATE_UNIX]
    ,OLD_DATE_UNIX as [OLD_DATE_UNIX]
FROM BASE_CTE
  WHERE RANK_DATE = 1 --Only the most recent record

#### [V_DIFFERENCES_MUTATIONS__PERCENTAGE_GAMMA]

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCES_MUTATIONS__PERCENTAGE_GAMMA]
AS
WITH BASE_CTE AS (
SELECT [DATE_START_UNIX]
      ,LAG(DATE_START_UNIX,1) OVER (ORDER BY DATE_START_UNIX) AS OLD_DATE_UNIX
      ,[GAMMA_PERCENTAGE] AS NEW_VALUE
      ,LAG(GAMMA_PERCENTAGE,1) OVER (ORDER BY DATE_START_UNIX) AS OLD_VALUE
      ,ROW_NUMBER() OVER (ORDER BY DATE_START_UNIX DESC) AS RANK_DATE
  FROM [VWSDEST].[V_RIVM_MUTATIONS]
  --Date_last_inserted filter is not needed since this is a view on a view.
)
SELECT  
     OLD_VALUE
    -- ,NEW_VALUE
    ,ROUND(NEW_VALUE - OLD_VALUE,1) AS [DIFFERENCE]
    ,DATE_START_UNIX AS [NEW_DATE_UNIX]
    ,OLD_DATE_UNIX as [OLD_DATE_UNIX]
FROM BASE_CTE
  WHERE RANK_DATE = 1 --Only the most recent record

#### [V_DIFFERENCES_MUTATIONS__PERCENTAGE_KAPPA]

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCES_MUTATIONS__PERCENTAGE_KAPPA]
AS
WITH BASE_CTE AS (
SELECT [DATE_START_UNIX]
      ,LAG(DATE_START_UNIX,1) OVER (ORDER BY DATE_START_UNIX) AS OLD_DATE_UNIX
      ,[KAPPA_PERCENTAGE] AS NEW_VALUE
      ,LAG(KAPPA_PERCENTAGE,1) OVER (ORDER BY DATE_START_UNIX) AS OLD_VALUE
      ,ROW_NUMBER() OVER (ORDER BY DATE_START_UNIX DESC) AS RANK_DATE
  FROM [VWSDEST].[V_RIVM_MUTATIONS]
  --Date_last_inserted filter is not needed since this is a view on a view.
)
SELECT  
     OLD_VALUE
    -- ,NEW_VALUE
    ,ROUND(NEW_VALUE - OLD_VALUE,1) AS [DIFFERENCE]
    ,DATE_START_UNIX AS [NEW_DATE_UNIX]
    ,OLD_DATE_UNIX as [OLD_DATE_UNIX]
FROM BASE_CTE
  WHERE RANK_DATE = 1 --Only the most recent record

#### [V_DIFFERENCES_MUTATIONS__PERCENTAGE_LAMBDA]

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCES_MUTATIONS__PERCENTAGE_LAMBDA]
AS
WITH BASE_CTE AS (
SELECT [DATE_START_UNIX]
      ,LAG(DATE_START_UNIX,1) OVER (ORDER BY DATE_START_UNIX) AS OLD_DATE_UNIX
      ,[LAMBDA_PERCENTAGE] AS NEW_VALUE
      ,LAG(LAMBDA_PERCENTAGE,1) OVER (ORDER BY DATE_START_UNIX) AS OLD_VALUE
      ,ROW_NUMBER() OVER (ORDER BY DATE_START_UNIX DESC) AS RANK_DATE
  FROM [VWSDEST].[V_RIVM_MUTATIONS]
  --Date_last_inserted filter is not needed since this is a view on a view.
)
SELECT  
     OLD_VALUE
    -- ,NEW_VALUE
    ,ROUND(NEW_VALUE - OLD_VALUE,1) AS [DIFFERENCE]
    ,DATE_START_UNIX AS [NEW_DATE_UNIX]
    ,OLD_DATE_UNIX as [OLD_DATE_UNIX]
FROM BASE_CTE
  WHERE RANK_DATE = 1 --Only the most recent record

#### [V_DIFFERENCES_MUTATIONS__PERCENTAGE_THETA]

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCES_MUTATIONS__PERCENTAGE_THETA]
AS
WITH BASE_CTE AS (
SELECT [DATE_START_UNIX]
      ,LAG(DATE_START_UNIX,1) OVER (ORDER BY DATE_START_UNIX) AS OLD_DATE_UNIX
      ,[THETA_PERCENTAGE] AS NEW_VALUE
      ,LAG(THETA_PERCENTAGE,1) OVER (ORDER BY DATE_START_UNIX) AS OLD_VALUE
      ,ROW_NUMBER() OVER (ORDER BY DATE_START_UNIX DESC) AS RANK_DATE
  FROM [VWSDEST].[V_RIVM_MUTATIONS]
  --Date_last_inserted filter is not needed since this is a view on a view.
)
SELECT  
     OLD_VALUE
    -- ,NEW_VALUE
    ,ROUND(NEW_VALUE - OLD_VALUE,1) AS [DIFFERENCE]
    ,DATE_START_UNIX AS [NEW_DATE_UNIX]
    ,OLD_DATE_UNIX as [OLD_DATE_UNIX]
FROM BASE_CTE
  WHERE RANK_DATE = 1 --Only the most recent record

#### [V_DIFFERENCES_MUTATIONS__PERCENTAGE_OTHER]

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCES_MUTATIONS__PERCENTAGE_OTHER]
AS
WITH BASE_CTE AS (
SELECT [DATE_START_UNIX]
      ,LAG(DATE_START_UNIX,1) OVER (ORDER BY DATE_START_UNIX) AS OLD_DATE_UNIX
      ,[OTHER_PERCENTAGE] AS NEW_VALUE
      ,LAG(OTHER_PERCENTAGE,1) OVER (ORDER BY DATE_START_UNIX) AS OLD_VALUE
      ,ROW_NUMBER() OVER (ORDER BY DATE_START_UNIX DESC) AS RANK_DATE
  FROM [VWSDEST].[V_RIVM_MUTATIONS]
  --Date_last_inserted filter is not needed since this is a view on a view.
)
SELECT  
     OLD_VALUE
    -- ,NEW_VALUE
    ,ROUND(NEW_VALUE - OLD_VALUE,1) AS [DIFFERENCE]
    ,DATE_START_UNIX AS [NEW_DATE_UNIX]
    ,OLD_DATE_UNIX as [OLD_DATE_UNIX]
FROM BASE_CTE
  WHERE RANK_DATE = 1 --Only the most recent record

# **Stored Procedures**
---

### STAGING TO INTER

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

CREATE OR ALTER PROCEDURE [dbo].[SP_RIVM_MUTATIONS_INTER]
AS
BEGIN
    INSERT INTO VWSINTER.RIVM_MUTATIONS
    (
       [VERSION],
       [DATE_OF_REPORT],
       [DATE_OF_STATISTICS_WEEK_START],
       [VARIANT_CODE],
       [VARIANT_NAME],
       [ECDC_CATEGORY],
       [SAMPLE_SIZE],
       [VARIANT_CASES]
    )
    SELECT
        CAST(ISNULL(NULLIF([VERSION], ''), 0) AS INT)               AS [VERSION],
        CONVERT(DATETIME, [DATE_OF_REPORT], 120)                    AS [DATE_OF_REPORT],
        CONVERT(DATETIME, [DATE_OF_STATISTICS_WEEK_START], 120)     AS [DATE_OF_STATISTICS_WEEK_START],
        [VARIANT_CODE],
        [VARIANT_NAME],
        [ECDC_CATEGORY],
        CAST(SAMPLE_SIZE AS INT),
        CAST(VARIANT_CASES AS INT)
    FROM 
       VWSSTAGE.RIVM_MUTATIONS
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) from VWSSTAGE.RIVM_MUTATIONS)
END;

### INTER TO DEST CALCULATIONS

In [ ]:
-- CREATE OR ALTER PROC dbo.SP_RECALC_VARIANT_CASES_SET
--     @lastDateMasterDataVars datetime
-- AS
-- BEGIN 
--     with 
--     T as
--     (
--         select
--             da.ID, da.VARIANT_CODE, da.VARIANT_CASES, vb.Is_subvariant_of,
--             isnull(vb.Shows_on_vws_dashboard, cast(0 as bit)) as Shows_on_vws_dashboard
--         from
--             ##VARIANT_CASES_SET as da
--             left join 
--             (SELECT * FROM VWSSTATIC.MASTERDATA_VARIANTS WHERE DATE_LAST_INSERTED = @lastDateMasterDataVars) as vb
--                 on da.VARIANT_CODE = vb.Variant_code
--     ),
--     U as
--     (
--         select
--             T.ID, T.VARIANT_CODE, T.VARIANT_CASES, T.Is_subvariant_of, T.Shows_on_vws_dashboard,
--             case T.Shows_on_vws_dashboard
--                 when 1 then T.VARIANT_CASES
--                 else 0 
--             end as VarCases,
--             0 as PrevShown
--         from T
--         where not exists(select * from T as t2 where Is_subvariant_of = T.VARIANT_CODE)

--         union all

--         select
--             T.ID, T.VARIANT_CODE, T.VARIANT_CASES, T.Is_subvariant_of, T.Shows_on_vws_dashboard,
--             case T.Shows_on_vws_dashboard
--                 when 1 then T.VARIANT_CASES - U.PrevShown
--                 else U.PrevShown
--             end as VarCases,
--             case U.Shows_on_vws_dashboard
--                 when 1 then U.VARIANT_CASES
--                 else U.PrevShown
--             end as PrevShown
--         from
--             U join T on U.Is_subvariant_of = T.VARIANT_CODE
--     )
--     select 
--         U.ID, U.VARIANT_CODE, U.VARIANT_CASES, U.Is_subvariant_of, U.Shows_on_vws_dashboard,
--         U.VARIANT_CASES - sum(PrevShown) as VarCases,
--         sum(PrevShown) as PrevShown
--     from U
--     group by U.ID, U.VARIANT_CODE, U.VARIANT_CASES, U.Is_subvariant_of, U.Shows_on_vws_dashboard;
-- END
-- GO

### (NEW) INTER TO DEST CALCULATIONS

In [ ]:
CREATE OR ALTER PROC dbo.SP_RECALC_VARIANT_CASES_SET
    @lastDateMasterDataVars datetime
AS
BEGIN 

with 
T as
(
	select
		da.VARIANT_CODE,
		da.VARIANT_CASES,
		vb.Is_subvariant_of,
		isnull(vb.Shows_on_vws_dashboard, cast(0 as bit)) as Shows_on_vws_dashboard
	from
		##VARIANT_CASES_SET da
		left join (SELECT * FROM VWSSTATIC.MASTERDATA_VARIANTS WHERE DATE_LAST_INSERTED = @lastDateMasterDataVars) as vb on da.VARIANT_CODE = vb.Variant_code
),
-- Fill in level for each node in tree
U as 
(
	select T.*, cast(1 as int) as [Level]
	from T
	where Is_subvariant_of is null

	union all
	
	select T.*, [Level] + 1 as [Level]
	from U join T on U.VARIANT_CODE = T.Is_subvariant_of
)
select *,
	cast(null as int) as VarCases,
	cast(0 as int) as Shown
into #Result
from U;

declare @level int;
select @level = max([Level]) from #Result;

-- Tree traversal from leafs to root,
-- one level each time through the loop
while @level > 0
begin
	with T as
	(
		select
			P.VARIANT_CODE, 
			case P.Shows_on_vws_dashboard
				when 1 then P.VARIANT_CASES - sum(isnull(C.Shown,0))
				else null
			end as VarCases,
			case
				when P.Shows_on_vws_dashboard = 1 then P.VARIANT_CASES
				else sum(case C.Shows_on_vws_dashboard
							when 1 then C.VARIANT_CASES
							else isnull(C.Shown, 0)
						 end)
			end as Shown
		from -- P:Parent, C:Child
			#Result as P
			left join #Result as C on P.VARIANT_CODE = C.Is_subvariant_of
		where 
			P.[Level] = @level
		group by
			P.VARIANT_CODE, P.VARIANT_CASES, P.Shows_on_vws_dashboard
	)
	update P
		set 
			VarCases =	T.VarCases,
			Shown = T.Shown
	from
		#Result as P
		join T on P.VARIANT_CODE = T.VARIANT_CODE;

	set @level -= 1;
end

select
	VARIANT_CODE, VARIANT_CASES, Is_subvariant_of, Shows_on_vws_dashboard, VarCases, Shown
from #Result


END
GO

### INTER TO DEST

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.
CREATE OR ALTER PROCEDURE [dbo].[SP_RIVM_MUTATIONS]
AS
BEGIN
    DECLARE
        @dateLastInserted DATETIME,
        @dateOfStatsStart DATETIME,
        @lastDateMasterDataVars DATETIME,
        @SAMPLE_SIZE INT,
        @TOT_VARIANT_CASES INT,
        @DateOfReport DATETIME,
        @InsertedDatedThisRun DATETIME = SYSDATETIME();

    DECLARE @RecalcResult TABLE
    (
        ID               INT,
        VARIANT_CODE     VARCHAR(100),
        VARIANT_CASES    INT,
        Is_subvariant_of VARCHAR(200),
        Shows_on_vws_dashboard BIT,
        VarCases           INT,
        PrevShown          INT,
        [Percentage]       NUMERIC(16,3)
    );

    SELECT @lastDateMasterDataVars = MAX(DATE_LAST_INSERTED) FROM VWSSTATIC.MASTERDATA_VARIANTS

    DROP TABLE IF EXISTS #LEVERINGEN;

    SELECT @dateLastInserted = MAX(DATE_LAST_INSERTED) FROM [VWSINTER].[RIVM_MUTATIONS];

    SELECT DISTINCT DATE_OF_STATISTICS_WEEK_START, CAST(0 AS BIT) AS DONE
    INTO #LEVERINGEN
    FROM [VWSINTER].[RIVM_MUTATIONS]
    WHERE DATE_LAST_INSERTED = @dateLastInserted;

    WHILE 1 = 1
    BEGIN
        SELECT TOP(1) @dateOfStatsStart = DATE_OF_STATISTICS_WEEK_START
        FROM #LEVERINGEN
        WHERE DONE = 0;

        IF @@ROWCOUNT = 0 BREAK;
        
        DROP TABLE IF EXISTS ##VARIANT_CASES_SET;

        SELECT  [ID], VARIANT_CODE, VARIANT_NAME, ECDC_CATEGORY, SAMPLE_SIZE, VARIANT_CASES
        INTO ##VARIANT_CASES_SET
        FROM [VWSINTER].[RIVM_MUTATIONS]
        WHERE DATE_LAST_INSERTED = @dateLastInserted AND DATE_OF_STATISTICS_WEEK_START = @dateOfStatsStart;

        DELETE @RecalcResult;

        INSERT @RecalcResult (ID, VARIANT_CODE, VARIANT_CASES, Is_subvariant_of, Shows_on_vws_dashboard, VarCases, PrevShown)
            EXEC dbo.SP_RECALC_VARIANT_CASES_SET @lastDateMasterDataVars = @lastDateMasterDataVars;

        SELECT TOP(1) @SAMPLE_SIZE = SAMPLE_SIZE FROM ##VARIANT_CASES_SET;

        SELECT @TOT_VARIANT_CASES = SUM(VarCases) FROM @RecalcResult WHERE Shows_on_vws_dashboard = 1 ;


        SELECT TOP(1) @DateOfReport = DATE_OF_REPORT
        FROM @RecalcResult AS R
             JOIN [VWSINTER].[RIVM_MUTATIONS] AS M ON R.ID = M.ID

        INSERT [VWSDEST].[RIVM_MUTATIONS]
                ([DATE_OF_REPORT], [DATE_OF_STATISTICS_WEEK_START], [VARIANT_CODE], [VARIANT_NAME], [LABEL_EN], [LABEL_NL], [SORT_ORDER], [IS_VARIANT_OF_CONCERN], [SAMPLE_SIZE], [VARIANT_CASES], [VARIANT_PERCENTAGE], [HAS_HISTORICAL_SIGNIFICANCE], [DATE_LAST_INSERTED])
            SELECT
                M.DATE_OF_REPORT,
                M.DATE_OF_STATISTICS_WEEK_START,
                M.VARIANT_CODE, 
                M.VARIANT_NAME,
                V.[LABEL_EN],
                V.[LABEL_NL],
                V.[SORT_ORDER],
                'true' AS IS_VARIANT_OF_CONCERN,
                M.SAMPLE_SIZE,
                R.VarCases as VARIANT_CASES,
                ROUND(CAST(VarCases * 100 AS NUMERIC(16,3)) / @SAMPLE_SIZE, 1) AS VARIANT_PERCENTAGE,
                IIF(M.[VARIANT_NAME] IN ('Alpha','Beta','Gamma'), 'true', 'false') AS [HAS_HISTORICAL_SIGNIFICANCE],
                @InsertedDatedThisRun
            FROM
                @RecalcResult AS R
                JOIN [VWSINTER].[RIVM_MUTATIONS] AS M ON R.ID = M.ID
                JOIN (SELECT * FROM VWSSTATIC.MASTERDATA_VARIANTS WHERE DATE_LAST_INSERTED = @lastDateMasterDataVars) AS V ON R.VARIANT_CODE = V.Variant_code
            WHERE
                R.Shows_on_vws_dashboard = 1
            
            UNION ALL

            SELECT
                @DateOfReport AS DATE_OF_REPORT,
                @dateOfStatsStart AS [DATE_OF_STATISTICS_WEEK_START],
                ''                                       AS [VARIANT_CODE],
                T.VarName                                AS [VARIANT_NAME],
                T.VarName                                AS [LABEL_EN],
                T.VarName                                AS [LABEL_NL],
                99                                       AS [SORT_ORDER],
                'true'                                  AS [IS_VARIANT_OF_CONCERN],
                @SAMPLE_SIZE                             AS [SAMPLE_SIZE],
                @SAMPLE_SIZE - @TOT_VARIANT_CASES        AS [VARIANT_CASES],
                ROUND((CAST((@SAMPLE_SIZE - @TOT_VARIANT_CASES) AS FLOAT)*100) /@SAMPLE_SIZE, 1) AS [VARIANT_PERCENTAGE],
                'false'                                  AS [HAS_HISTORICAL_SIGNIFICANCE],
                @InsertedDatedThisRun                    AS [DATE_LAST_INSERTED]
            FROM 
                (VALUES
                    ('other_table'),
                    ('other_graph') ) as T(VarName);

        DROP TABLE IF EXISTS ##VARIANT_CASES_SET;

        UPDATE #LEVERINGEN
            SET DONE = 1
        WHERE DATE_OF_STATISTICS_WEEK_START = @dateOfStatsStart;

    END
END
GO

### INTER TO DEST (OLD)

In [ ]:
-- -- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- -- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.
-- ALTER   PROCEDURE [dbo].[SP_RIVM_MUTATIONS]
-- AS
-- BEGIN

-- WITH BASE_CTE AS (
-- SELECT
--        [DATE_OF_REPORT],
--        [DATE_OF_STATISTICS_WEEK_START],
--        [VARIANT_CODE],
--        [VARIANT_NAME],
--        [ECDC_CATEGORY],
--        IIF([ECDC_CATEGORY] = 'VOC' 
--             OR [VARIANT_CODE] = 'B.1.1.529'
--             OR [VARIANT_NAME] IN ('Alpha', 'Beta','Gamma'), 'true','false') AS [IS_VARIANT_OF_CONCERN],
--        IIF([VARIANT_NAME] IN ('Alpha','Beta','Gamma'), 'true', 'false') AS [HAS_HISTORICAL_SIGNIFICANCE],
--        [SAMPLE_SIZE],
--        [VARIANT_CASES],
--        CAST([VARIANT_CASES] AS FLOAT) / [SAMPLE_SIZE] AS [VARIANT_PERCENTAGE],
--        [DATE_LAST_INSERTED]
--   FROM [VWSINTER].[RIVM_MUTATIONS]
--   WHERE
--        DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSINTER].[RIVM_MUTATIONS]) AND
--        VARIANT_NAME <> '' AND (
--        ECDC_CATEGORY IN ('VOC', 'VOI')
--        OR VARIANT_NAME IN (
--                 'Alpha'
--                 ,'Omicron'     -- toevoeging
--                 ,'Beta'
--                 ,'Gamma'
--        --        ,'Delta'
--        --        ,'Epsilon'
--        --        ,'Eta'
--        --        ,'Theta'
--        --        ,'Kappa'
--        --        --,'Lambda'
--        )
--        )
-- )
-- ,
-- --Use a grid with all valid date and variant combinations
-- --to cover the situation that a variant is not present in all weeks
-- GRID_CTE AS (

--     SELECT DISTINCT
--          T0.[DATE_OF_STATISTICS_WEEK_START]
--         ,T1.VARIANT_CODE
--         FROM BASE_CTE AS T0
--             CROSS JOIN
--                 (SELECT DISTINCT [VARIANT_CODE] FROM BASE_CTE) AS T1
-- ),
-- FULL_CTE AS --Grid is filled with all present data
-- (
-- SELECT
--     T0.[DATE_OF_STATISTICS_WEEK_START],
--     T0.VARIANT_CODE,
--     T1.[DATE_OF_REPORT],
--     T1.[VARIANT_NAME],
--     T1.[ECDC_CATEGORY],
--     T1.[IS_VARIANT_OF_CONCERN],
--     T1.[HAS_HISTORICAL_SIGNIFICANCE],
--     T1.[SAMPLE_SIZE],
--     T1.[VARIANT_CASES],
--     T1.[VARIANT_PERCENTAGE],
--     T1.[DATE_LAST_INSERTED]
--   FROM GRID_CTE AS T0
--     LEFT JOIN BASE_CTE AS T1 ON T0.[DATE_OF_STATISTICS_WEEK_START] = T1.[DATE_OF_STATISTICS_WEEK_START] AND T0.VARIANT_CODE = T1.VARIANT_CODE
-- )
-- ,
-- TOTALS_CTE AS (
-- SELECT
--        [DATE_OF_REPORT],
--        [DATE_OF_STATISTICS_WEEK_START],
--        MAX([SAMPLE_SIZE])                       AS [SAMPLE_SIZE],
--        SUM([VARIANT_CASES])                     AS [VARIANT_CASES]

--     FROM FULL_CTE
--     WHERE ECDC_CATEGORY IN ('VOC', 'VOI') OR [VARIANT_CODE] = 'B.1.1.529'
--            OR VARIANT_NAME IN (
--                 'Alpha'
--                 ,'Omicron'     -- toevoeging
--                 ,'Beta'
--                 ,'Gamma'
--        --        ,'Delta'
--        --        ,'Epsilon'
--        --        ,'Eta'
--        --        ,'Theta'
--        --        ,'Kappa'
--        --        --,'Lambda'
--        )
--     GROUP BY [DATE_OF_REPORT], [DATE_OF_STATISTICS_WEEK_START]
-- )
-- ,
-- TOTALS_CTE2 AS (
-- SELECT
--         [DATE_OF_REPORT],
--         [DATE_OF_STATISTICS_WEEK_START],
--         MAX([SAMPLE_SIZE])              AS [SAMPLE_SIZE],
--         SUM([VARIANT_CASES])            AS [VARIANT_CASES]


--      FROM FULL_CTE
--      WHERE ECDC_CATEGORY IN ('VOC') or HAS_HISTORICAL_SIGNIFICANCE = 'true' OR [VARIANT_CODE] = 'B.1.1.529'
--      GROUP BY [DATE_OF_REPORT], [DATE_OF_STATISTICS_WEEK_START]
-- )

-- INSERT INTO VWSDEST.RIVM_MUTATIONS
--       (
--          [DATE_OF_REPORT],
--          [DATE_OF_STATISTICS_WEEK_START],
--          [VARIANT_CODE],
--          [VARIANT_NAME],
--          [IS_VARIANT_OF_CONCERN],
--          [HAS_HISTORICAL_SIGNIFICANCE],
--          [SAMPLE_SIZE],
--          [VARIANT_CASES],
--          [VARIANT_PERCENTAGE]
--       )

-- SELECT
--        [DATE_OF_REPORT],
--        [DATE_OF_STATISTICS_WEEK_START],
--        [VARIANT_CODE],
--        [VARIANT_NAME],
--        [IS_VARIANT_OF_CONCERN],
--        [HAS_HISTORICAL_SIGNIFICANCE],
--        [SAMPLE_SIZE],
--        [VARIANT_CASES],
--        ROUND(CAST([VARIANT_CASES]*100 AS FLOAT) / [SAMPLE_SIZE],1) AS [VARIANT_PERCENTAGE]
--   FROM [FULL_CTE]
-- UNION ALL
-- --other variants tabel
-- SELECT
--        [DATE_OF_REPORT],
--        [DATE_OF_STATISTICS_WEEK_START],
--        ''                                       AS [VARIANT_CODE],
--        'Other_Table'                            AS [VARIANT_NAME],
--        'false'                                  AS [IS_VARIANT_OF_CONCERN],
--        'false'                                  AS [HAS_HISTORICAL_SIGNIFICANCE],
--        [SAMPLE_SIZE],
--        [SAMPLE_SIZE] - [VARIANT_CASES]          AS [VARIANT_CASES],
--        ROUND((CAST(([SAMPLE_SIZE] - [VARIANT_CASES]) AS FLOAT)*100) / [SAMPLE_SIZE],1) AS [VARIANT_PERCENTAGE]

--  FROM TOTALS_CTE
-- UNION ALL
--  --other variants grafiek
-- SELECT
--        [DATE_OF_REPORT],
--        [DATE_OF_STATISTICS_WEEK_START],
--        ''                                       AS [VARIANT_CODE],
--        'Other_Graph'                          AS [VARIANT_NAME],
--        'false'                                  AS [IS_VARIANT_OF_CONCERN],
--        'true'                                  AS [HAS_HISTORICAL_SIGNIFICANCE],
--        [SAMPLE_SIZE],
--        [SAMPLE_SIZE] - [VARIANT_CASES]          AS [VARIANT_CASES],
--        ROUND((CAST(([SAMPLE_SIZE] - [VARIANT_CASES]) AS FLOAT)*100) / [SAMPLE_SIZE],1) AS [VARIANT_PERCENTAGE]

--  FROM TOTALS_CTE2
-- END
-- GO


# **DATATINO CONFIGURATIONS**
---

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'RIVM_MUTATIONS',
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;

IF NOT EXISTS ( SELECT 1 FROM [DATATINO_ORCHESTRATOR_1].[DATAFLOWS] WHERE NAME LIKE @workflow_name )
BEGIN
    -- 1.1) SET ENVIRONMENTAL VARIABLES.....
    SET @is_active = CASE LOWER('#{ Environment }#')
        WHEN 'production' THEN 1
        WHEN 'acceptance' THEN 1
        ELSE 1
    END;

    SELECT TOP(1)
        @workflow_id = workflows.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
    WHERE v_workflows.[NAME] = @workflow_name;

    SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_name);

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
        @id = @workflow_id, 
        @workflow_name = @workflow_name,
        @description = @workflow_description,
        @schedule = '0 12 * * MON-FRI', -- At 01:00 PM, mondays to fridays
        @active = @is_active;

    -- 2) UPSERT SOURCE(S).....
    DECLARE @source NVARCHAR(256),
            @source_id INT,
            @source_name NVARCHAR(256),
            @source_description NVARCHAR(256),
            @target_name VARCHAR(256),
            @location_type VARCHAR(50),
            @security_profile VARCHAR(50);

    -- 2.1) SET ENVIRONMENTAL VARIABLES.....
    SET @source = CASE LOWER('#{ Environment }#')
        WHEN 'production' THEN 'https://data.rivm.nl/data/vws/covid-19/COVID-19_varianten.csv'
        WHEN 'acceptance' THEN 'https://acceptatie.data.rivm.nl/data/vws/covid-19/COVID-19_varianten.csv'
        ELSE 'https://acceptatie.data.rivm.nl/data/vws/covid-19/COVID-19_varianten.csv'
    END;

    SET @location_type = CASE LOWER('#{ Environment }#')
        WHEN 'production' THEN 'Web'
        WHEN 'acceptance' THEN 'Web'
        ELSE 'Web'
    END;

    SET @security_profile = CASE LOWER('#{ Environment }#')
        WHEN 'production' THEN 'RIVM'
        WHEN 'acceptance' THEN 'RIVM'
        ELSE 'RIVM'
    END;

    -- 2.2) UPSERT TABLE SOURCE(S): STAGING......
    SET @source_name = CONCAT('SOURCE_', @workflow_name);
    SET @source_description = CONCAT('LOAD: ', @workflow_name);
    SET @target_name = CONCAT('VWSSTAGE.', @workflow_name);

    SELECT TOP(1)
        @source_id = [ID]
    FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
    WHERE [NAME] = @source_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
        @id = @source_id,
        @source_name = @source_name,
        @description = @source_description,
        @source = @source,
        @source_columns = 'Version|Date_of_report|Date_of_statistics_week_start|Variant_code|Variant_name|ECDC_category|Sample_size|Variant_cases',
        @target_columns = 'VERSION|DATE_OF_REPORT|DATE_OF_STATISTICS_WEEK_START|VARIANT_CODE|VARIANT_NAME|ECDC_CATEGORY|SAMPLE_SIZE|VARIANT_CASES',
        @target_name = @target_name,
        @source_type = 'CsvFile',
        @location_type = @location_type,
        @delimiter_type = 'SemiColon',
        @security_profile= @security_profile;

    -- 2.3) UPSERT STORED PROCEDURE SOURCE(S): STAGE TO INTER......
    SET @source = CONCAT('dbo.SP_', @workflow_name, '_STAGE_TO_INTER');
    SET @source_name = CONCAT('SOURCE_SP_', @workflow_name, '_STAGE_TO_INTER');
    SET @source_description = CONCAT('MAP: ', @workflow_name, ' FROM STAGE TO INTER');

    SELECT TOP(1)
        @source_id = [ID]
    FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
    WHERE [NAME] = @source_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
        @id = @source_id,
        @source_name = @source_name,
        @description = @source_description,
        @source = @source,
        @source_columns = null,
        @target_columns = null,
        @target_name = null,
        @source_type = 'StoredProcedure',
        @location_type = 'N/A',
        @delimiter_type = 'N/A',
        @security_profile= @security_profile;

    -- 2.4) UPSERT STORED PROCEDURE SOURCE(S): INTER TO DEST.....
    SET @source = CONCAT('dbo.SP_', @workflow_name, '_INTER_TO_DEST');
    SET @source_name = CONCAT('SOURCE_SP_', @workflow_name, '_INTER_TO_DEST');
    SET @source_description = CONCAT('MAP: ', @workflow_name, ' FROM INTER TO DEST');

    SELECT TOP(1)
        @source_id = [ID]
    FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
    WHERE [NAME] = @source_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
        @id = @source_id,
        @source_name = @source_name,
        @description = @source_description,
        @source = @source,
        @source_columns = null,
        @target_columns = null,
        @target_name = null,
        @source_type = 'StoredProcedure',
        @location_type = 'N/A',
        @delimiter_type = 'N/A',
        @security_profile= @security_profile;

    -- 3) UPSERT PROCESS(ES).....
    DECLARE @process_id INT,
            @process_name NVARCHAR(256),
            @process_description NVARCHAR(256),
            @process_source_name VARCHAR(256);

    -- 3.1) UPSERT TABLE PROCESS(ES): STAGING......
    SET @process_name = CONCAT('PROCESS_', @workflow_name);
    SET @process_description = CONCAT('LOAD: ', @workflow_name);
    SET @process_source_name = CONCAT('SOURCE_', @workflow_name);

    SELECT TOP(1)
        @process_id = processes.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
    WHERE v_processes.[PROCESS_NAME] = @process_name 
        AND v_workflows.[NAME] = @workflow_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
        @id = @process_id,
        @process_name = @process_name,
        @description = @process_description,
        @source_name = @process_source_name,
        @schedule = '* * * * *',
        @workflow_name = @workflow_name,
        @active = 1;

    -- 3.2) UPSERT STORED PROCEDURE PROCESS(S): STAGE TO INTER......
    SET @process_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGE_TO_INTER');
    SET @process_description = CONCAT('MAP: ', @workflow_name, ' FROM STAGE TO INTER');
    SET @process_source_name = CONCAT('SOURCE_SP_', @workflow_name, '_STAGE_TO_INTER');

    SELECT TOP(1)
        @process_id = processes.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
    WHERE v_processes.[PROCESS_NAME] = @process_name 
        AND v_workflows.[NAME] = @workflow_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
        @id = @process_id,
        @process_name = @process_name,
        @description = @process_description,
        @source_name = @process_source_name,
        @schedule = '* * * * *',
        @workflow_name = @workflow_name,
        @active = 1;

    -- 3.3) UPSERT STORED PROCEDURE PROCESS(S): INTER TO DEST......
    SET @process_name = CONCAT('PROCESS_SP_', @workflow_name, '_INTER_TO_DEST');
    SET @process_description = CONCAT('MAP: ', @workflow_name, ' FROM INTER TO DEST');
    SET @process_source_name = CONCAT('SOURCE_SP_', @workflow_name, '_INTER_TO_DEST');

    SELECT TOP(1)
        @process_id = processes.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
    WHERE v_processes.[PROCESS_NAME] = @process_name 
        AND v_workflows.[NAME] = @workflow_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
        @id = @process_id,
        @process_name = @process_name,
        @description = @process_description,
        @source_name = @process_source_name,
        @schedule = '* * * * *',
        @workflow_name = @workflow_name,
        @active = 1;

    -- 4) UPSERT DEPENDENC(Y)(IES).....
    DECLARE @dependency_id INT,
            @dependency_name NVARCHAR(256),
            @dependency_description NVARCHAR(256),
            @dependency_dataflow_name NVARCHAR(256),
            @dependency_process_name NVARCHAR(256);

    -- 4.1) UPSERT STAGING TO STORE PROCEDURE DEPENDENC(Y)(IES).....
    SET @dependency_name = CONCAT('DEPENDENCY_', @workflow_name,'_STAGE_TO_INTER');
    SET @dependency_dataflow_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGE_TO_INTER');
    SET @dependency_process_name = CONCAT('PROCESS_', @workflow_name);
    SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

    SELECT TOP(1)
        @dependency_id = dependencies.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
    WHERE dependencies.[NAME] = @dependency_name
        AND v_dependencies.[WORKFLOW_NAME] = @workflow_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
        @id = @dependency_id,
        @dataflow_name = @dependency_dataflow_name,
        @dataflowtype_id = 2,
        @dependency_name = @dependency_process_name,
        @dependencytype_id = 2,
        @workflow_name = @workflow_name,
        @name = @dependency_name,
        @description = @dependency_description,
        @active = 1;

    -- 4.2) UPSERT STORED PROCEDURE TO STORED PROCEDURE DEPENDENC(Y)(IES)......
    SET @dependency_name = CONCAT('DEPENDENCY_', @workflow_name,'_INTER_TO_DEST');
    SET @dependency_dataflow_name = CONCAT('PROCESS_SP_', @workflow_name, '_INTER_TO_DEST');
    SET @dependency_process_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGE_TO_INTER');
    SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

    SELECT TOP(1)
        @dependency_id = dependencies.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
    WHERE dependencies.[NAME] = @dependency_name
        AND v_dependencies.[WORKFLOW_NAME] = @workflow_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
        @id = @dependency_id,
        @dataflow_name = @dependency_dataflow_name,
        @dataflowtype_id = 2,
        @dependency_name = @dependency_process_name,
        @dependencytype_id = 2,
        @workflow_name = @workflow_name,
        @name = @dependency_name,
        @description = @dependency_description,
        @active = 1;

END

### PROTO CONFIGURATIONS

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_MUTATIONS_TABULAR',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'variants',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = 'NAME',
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = 'DATE_START_UNIX|DATE_END_UNIX|VARIANT_CODE|VARIANT_NAME_LABEL_EN|INDEX|IS_VARIANT_OF_CONCERN|HAS_HISTORICAL_SIGNIFICANCE|SAMPLE_SIZE|OCCURRENCE|PERCENTAGE|DATE_OF_INSERTION_UNIX',
        @layout_type_id INT = 1,
        @last_update_name VARCHAR(50) = 'DATE_START_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....

PRINT 'PROTO NAME ->' + @proto_name

EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO